# Materials Project Deep Research - Lab Manager Agent (v2.1)

## Enhancements in This Version
- ✅ **LangSmith Integration** for observability and debugging
- ✅ **Crystal-specific test cases** (symmetry, coordination, bonding)
- ✅ **Structure visualization** using pymatgen plotting
- ✅ **Fixed MatGL backend** configuration

## Architecture Overview

```
┌─────────────────────────────────────────────────────────────┐
│                    LAB MANAGER AGENT                        │
│         (Discovery + Innovation + Visualization)           │
└─────────────────────────────────────────────────────────────┘
                            │
        ┌───────────────────┼───────────────────┐
        ▼                   ▼                   ▼
┌──────────────┐    ┌──────────────┐    ┌──────────────┐
│  DISCOVERY   │    │  INNOVATION  │    │ VISUALIZATION│
├──────────────┤    ├──────────────┤    ├──────────────┤
│ batch_screen │    │ substitute_  │    │ plot_struct  │
│ analyze_cand │    │ relax_m3gnet │    │ plot_phase_  │
│ phase_diag   │    │ assess_stab  │    │ plot_dos     │
└──────────────┘    └──────────────┘    └──────────────┘
```

## 0. Environment Setup with LangSmith

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Load secrets
load_dotenv()

# Ensure we can import the local package
sys.path.insert(0, "../src")

# =============================================================================
# LANGSMITH CONFIGURATION - For Observability
# =============================================================================
# Set these in your .env file:
#   LANGCHAIN_TRACING_V2=true
#   LANGSMITH_API_KEY=your-langsmith-api-key
#   LANGCHAIN_PROJECT=mp-deep-research

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2", "true")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT", "mp-deep-research-innovation")

# Check LangSmith status
langsmith_enabled = os.environ.get("LANGSMITH_API_KEY") is not None
if langsmith_enabled:
    print("🔍 LangSmith: ENABLED")
    print(f"   Project: {os.environ.get('LANGCHAIN_PROJECT')}")
    print("   View traces at: https://smith.langchain.com")
else:
    print("⚠️  LangSmith: DISABLED (set LANGSMITH_API_KEY in .env)")

# =============================================================================
# VERIFY API KEYS
# =============================================================================
assert os.environ.get("MP_API_KEY"), "MP_API_KEY required in .env"
assert os.environ.get("OPENAI_API_KEY"), "OPENAI_API_KEY required in .env"

# Setup Directories
CACHE_DIR = Path("../cache")
OUTPUT_DIR = Path("../outputs")
CACHE_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)

print("\n✓ Environment configured")
print(f"  Cache: {CACHE_DIR.resolve()}")
print(f"  Output: {OUTPUT_DIR.resolve()}")

🔍 LangSmith: ENABLED
   Project: mp-deep-research
   View traces at: https://smith.langchain.com

✓ Environment configured
  Cache: /Users/snnair/Documents/MP/mp_deep_research/cache
  Output: /Users/snnair/Documents/MP/mp_deep_research/outputs


## 1. Fix MatGL Backend (CRITICAL)

In [2]:
# =============================================================================
# MATGL BACKEND FIX - Must run before importing tools
# =============================================================================
os.environ["DGLBACKEND"] = "pytorch"  # Set before importing DGL

try:
    import torch
    import matgl
    
    print(f"PyTorch Version: {torch.__version__}")
    print(f"MatGL Version: {matgl.__version__}")
    
    # CRITICAL: Use uppercase "DGL" not "dgl"
    matgl.set_backend("DGL")
    print("\n✅ MatGL backend set to DGL")
    
    # Test model loading
    model = matgl.load_model("M3GNet-MP-2021.2.8-PES")
    print("✅ M3GNet model loaded successfully")
    del model  # Free memory
    
except Exception as e:
    print(f"❌ MatGL Error: {e}")
    print("\nTry: pip install matgl dgl -U")

PyTorch Version: 2.3.0
MatGL Version: 2.0.6

✅ MatGL backend set to DGL


/Users/snnair/miniconda3/envs/mp_research/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ M3GNet model loaded successfully


## 2. Load Tools with Inventory Check

In [3]:
from mp_deep_research.tools import ALL_TOOLS

# Create tool inventory
tool_names = [t.name for t in ALL_TOOLS]
print(f"✓ Loaded {len(ALL_TOOLS)} tools\n")

# Categorize by function
categories = {
    "📊 DISCOVERY": ["batch_screen_materials", "analyze_candidate", "identify_data_gaps"],
    "🔬 INNOVATION": ["substitute_species", "relax_structure_m3gnet", "assess_stability"],
    "📈 VISUALIZATION": ["plot_structure", "plot_phase_diagram", "plot_dos", "plot_bandstructure"],
    "🔍 ANALYSIS": ["compare_structures", "correlate_properties", "generate_phase_diagram"],
    "📚 LITERATURE": ["search_arxiv", "check_literature"],
    "🛠️ UTILITY": ["think"],
}

for category, expected_tools in categories.items():
    print(f"{category}:")
    for tool in expected_tools:
        status = "✅" if tool in tool_names else "❌ MISSING"
        print(f"  {status} {tool}")
    print()

# Warn about missing critical tools
missing = [t for cat in categories.values() for t in cat if t not in tool_names]
if missing:
    print(f"⚠️  Missing tools: {missing}")
    print("   Add these to tools/__init__.py")

✓ Loaded 10 tools

📊 DISCOVERY:
  ✅ batch_screen_materials
  ✅ analyze_candidate
  ❌ MISSING identify_data_gaps

🔬 INNOVATION:
  ✅ substitute_species
  ✅ relax_structure_m3gnet
  ✅ assess_stability

📈 VISUALIZATION:
  ❌ MISSING plot_structure
  ❌ MISSING plot_phase_diagram
  ❌ MISSING plot_dos
  ❌ MISSING plot_bandstructure

🔍 ANALYSIS:
  ✅ compare_structures
  ✅ correlate_properties
  ✅ generate_phase_diagram

📚 LITERATURE:
  ✅ search_arxiv
  ❌ MISSING check_literature

🛠️ UTILITY:
  ✅ think

⚠️  Missing tools: ['identify_data_gaps', 'plot_structure', 'plot_phase_diagram', 'plot_dos', 'plot_bandstructure', 'check_literature']
   Add these to tools/__init__.py


In [ ]:
# src/mp_deep_research/agent.py

import os
from typing import Annotated, Literal, TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

# Import your new "Super Tool"
from .tools.flows import run_innovation_flow
# Import standard lookups if you still want them
from .tools.standard import analyze_candidate, batch_screen_materials 

from datetime import datetime
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

# Optional: LangSmith callbacks for detailed tracing
try:
    from langsmith import Client
    from langchain_core.tracers import LangChainTracer
    
    tracer = LangChainTracer(project_name=os.environ.get("LANGCHAIN_PROJECT", "default"))
    callbacks = [tracer]
    print("✓ LangSmith tracer initialized")
except ImportError:
    callbacks = []
    print("⚠️ LangSmith not available - install with: pip install langsmith")

# --- ENHANCED PROMPT WITH CRYSTAL CHEMISTRY ---
# src/mp_deep_research/prompts.py

LAB_MANAGER_SYSTEM_PROMPT = """
You are a Lab Manager for computational materials research.

### TOOLS
You have access to a specialized workflow tool:
- `run_innovation_flow`: Use this for ANY request to "substitute", "create", "test", or "innovate" a new material.
  - Arguments: `material_id` (e.g., "mp-19017") and `substitutions` (e.g., {"Li": "Na"}).
  - This tool AUTOMATICALLY runs the full sequence (Theory → Substitution → Relaxation → Stability).
  - You do NOT need to call relax or assess_stability manually.

### OPERATIONAL RULES
1. **Always use the flow:** If a user asks to "make NaFePO4", just call `run_innovation_flow`.
2. **Trust the Report:** The tool will generate a detailed scientific report. Present this report to the user directly.
3. **No Math:** Do not attempt to convert units or calculate volume changes yourself. The tool does it.
"""

# 1. Define State
class AgentState(TypedDict):
    # 'messages' will store the chat history and tool outputs
    messages: Annotated[list, add_messages]

# 2. Setup Tools & LLM
# We mix standard tools (lookup) with flow tools (innovation)
tools = [
    analyze_candidate,        # Standard tool for simple lookups
    batch_screen_materials,   # Standard tool for screening
    run_innovation_flow       # <--- YOUR NEW SUPER TOOL
]

# Bind tools to the LLM (GPT-4o is recommended for complex function calling)
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# 3. Define Nodes
def chatbot(state: AgentState):
    """The brain: Decides whether to chat or call a tool."""
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 4. Define Graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", chatbot)
workflow.add_node("tools", ToolNode(tools)) # specific node to run Python functions

# Add edges
workflow.set_entry_point("agent")

# Conditional Logic: If LLM calls a tool, go to 'tools'. Otherwise, END.
def should_continue(state: AgentState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the LLM generated a tool_call, go to the tool execution node
    if last_message.tool_calls:
        return "tools"
    return END

workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent") # Loop back to agent to read the tool output

# Compile the graph
app = workflow.compile()

## 3. Build Agent with LangSmith Tracing

In [ ]:


# --- STATE ---
class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | SystemMessage | AIMessage], add_messages]

# --- MODEL ---
MODEL_NAME = "gpt-4o-mini"

def agent_node(state: AgentState) -> dict:
    model = ChatOpenAI(model=MODEL_NAME, temperature=0)
    model_with_tools = model.bind_tools(ALL_TOOLS)
    
    sys_msg = SystemMessage(content=LAB_MANAGER_PROMPT.format(
        date=datetime.now().strftime("%Y-%m-%d")
    ))
    
    response = model_with_tools.invoke([sys_msg] + list(state["messages"]))
    return {"messages": [response]}

def should_continue(state: AgentState) -> str:
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools"
    return END

# --- BUILD GRAPH ---
builder = StateGraph(AgentState)
builder.add_node("agent", agent_node)
builder.add_node("tools", ToolNode(ALL_TOOLS))
builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", should_continue)
builder.add_edge("tools", "agent")

lab_manager = builder.compile()

print(f"\n✓ Agent compiled with {len(ALL_TOOLS)} tools")
print(f"  Model: {MODEL_NAME}")

✓ LangSmith tracer initialized

✓ Agent compiled with 10 tools
  Model: gpt-4o-mini


## 4. Helper Function with Tool Tracking

In [9]:
from collections import defaultdict

# Track tool usage across all queries
TOOL_USAGE_LOG = defaultdict(int)

# Cell 4 or 5 in your notebook - the run_query helper function

def run_query(query: str, verbose: bool = True, max_iterations: int = 18) -> dict:
    """
    Run a query through the Lab Manager agent with loop protection.
    
    Args:
        query: The research question
        verbose: Print tool calls as they happen
        max_iterations: Maximum tool calls before forced stop (default: 12)
    """
    from collections import Counter
    
    print(f"\n{'='*60}")
    print(f"QUERY: {query[:80]}{'...' if len(query) > 80 else ''}")
    print(f"{'='*60}\n")
    
    final_response = ""
    tools_called = []
    tool_counter = Counter()
    
    try:
        # =====================================================
        # ADD recursion_limit HERE - this is the key line
        # =====================================================
        for event in lab_manager.stream(
            {"messages": [HumanMessage(content=query)]},
            {"recursion_limit": max_iterations}  # ← ADD THIS
        ):
            for node_name, node_output in event.items():
                if "messages" not in node_output:
                    continue
                    
                msg = node_output["messages"][-1]
                
                # Tool calls
                if hasattr(msg, "tool_calls") and msg.tool_calls:
                    for tc in msg.tool_calls:
                        tool_name = tc['name']
                        tool_args = str(tc['args'])
                        call_key = f"{tool_name}:{tool_args[:50]}"
                        
                        tools_called.append(tool_name)
                        tool_counter[call_key] += 1
                        
                        # Warn on repeated identical calls
                        if tool_counter[call_key] >= 2:
                            print(f"⚠️  REPEAT: {tool_name} ({tool_counter[call_key]}x same args)")
                        
                        if verbose:
                            print(f"🛠️  [{len(tools_called)}] {tool_name}")
                            if tool_name in ['substitute_species', 'analyze_candidate']:
                                print(f"    Args: {tc['args']}")
                
                # Tool results
                elif hasattr(msg, "content") and node_name == "tools":
                    if verbose and msg.content:
                        preview = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
                        print(f"    📄 {preview}")
                
                # Final response
                elif hasattr(msg, "content") and msg.content and node_name == "agent":
                    if not (hasattr(msg, "tool_calls") and msg.tool_calls):
                        final_response = msg.content
                        
    except Exception as e:
        error_msg = str(e).lower()
        if "recursion" in error_msg or "limit" in error_msg:
            print(f"\n⛔ Hit max iterations ({max_iterations}). Forcing stop.")
            final_response = f"Reached tool call limit. Summary of {len(tools_called)} calls made."
        else:
            print(f"\n❌ Error: {e}")
            raise
    
    # Summary
    print(f"\n{'─'*60}")
    print(f"COMPLETE: {len(tools_called)} tool calls")
    print(f"Tools used: {dict(Counter(tools_called))}")
    print(f"{'─'*60}")
    print(final_response)
    
    return {
        "response": final_response,
        "tools_called": tools_called,
        "call_count": len(tools_called)
    }

def show_tool_usage_summary():
    """Display summary of all tools called across queries."""
    print("\n" + "="*60)
    print("TOOL USAGE SUMMARY")
    print("="*60)
    
    if not TOOL_USAGE_LOG:
        print("No tools called yet.")
        return
    
    for tool, count in sorted(TOOL_USAGE_LOG.items(), key=lambda x: -x[1]):
        bar = "█" * count
        print(f"  {tool:30} {bar} ({count})")
    
    print(f"\nTotal tool calls: {sum(TOOL_USAGE_LOG.values())}")

---

## 5. Test Case 1: Crystal Structure Analysis

Test the agent's understanding of crystal symmetry and coordination.

In [ ]:
crystal_query = """
Analyze the crystal structure of spinel LiMn2O4 (mp-18767):

1. What is its space group and crystal system?
2. What are the coordination environments of Li and Mn?
3. Why is this structure good for Li-ion battery cathodes?

If you have visualization tools, show me the structure.
"""

result1 = run_query(crystal_query)

---

## 6. Test Case 2: Ionic Radius Substitution Logic

Test if the agent considers ionic radii when substituting.

In [ ]:
radius_query = """
I want to substitute Li with Na in the olivine structure LiFePO4 (mp-19017).

Before doing the substitution:
1. Compare the ionic radii of Li+ and Na+ in their typical coordination
2. Predict if the structure will expand, contract, or distort
3. Explain potential issues with this substitution

Then perform the substitution and relax with M3GNet to see what actually happens.
Compare the predicted vs actual volume change.
"""

result2 = run_query(radius_query)


QUERY: 
I want to substitute Li with Na in the olivine structure LiFePO4 (mp-19017).

B...

🛠️  [1] analyze_candidate
    Args: {'material_id': 'mp-19017'}
🛠️  [2] think


Retrieving SummaryDoc documents: 100%|██████████| 1/1 [00:00<00:00, 23301.69it/s]
/Users/snnair/miniconda3/envs/mp_research/lib/python3.11/site-packages/pymatgen/analysis/local_env.py:4021: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r1 = _get_radius(structure[n])
/Users/snnair/miniconda3/envs/mp_research/lib/python3.11/site-packages/pymatgen/analysis/local_env.py:4023: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r2 = _get_radius(entry["site"])
/Users/snnair/miniconda3/envs/mp_research/lib/python3.11/site-packages/pymatgen/analysis/local_env.py:3935: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  nn_data = self.get_nn_data(structure, n)


    📄 Thought recorded: Comparing ionic radii: Li+ typically has an ionic radius of 0.76 Å in octahedral coordination, while Na+ has an ionic radius of 1.02...
🛠️  [3] substitute_species
    Args: {'material_id': 'mp-19017', 'substitutions': {'Li': 'Na'}}


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 18808.54it/s]


    📄 
SUBSTITUTION SUCCESSFUL:
Parent Material: mp-19017 (LiFePO4)
New Composition: NaFePO4
Substitution...
🛠️  [4] relax_structure_m3gnet
    📄 
SIMULATION RESULTS (M3GNet + UQ):
Input: outputs/mp-19017_sub_Na.json
Final Composition: NaFePO4

...
🛠️  [5] assess_stability


Retrieving ThermoDoc documents: 100%|██████████| 557/557 [00:00<00:00, 14693253.64it/s]
/Users/snnair/miniconda3/envs/mp_research/lib/python3.11/site-packages/pymatgen/entries/computed_entries.py:369: FutureWarning: AffineScalarFunc.__bool__() is deprecated. In future releases it will defer to object.__bool__() and always return True.
  corr = sum(ufloat(ea.value, ea.uncertainty) for ea in self.energy_adjustments if ea.value) or ufloat(


    📄 
STABILITY ASSESSMENT:
Candidate: NaFePO4
Energy: -6.6842 eV/atom

RESULTS:
  Energy Above Hull: 0....

────────────────────────────────────────────────────────────
COMPLETE: 5 tool calls
Tools used: {'analyze_candidate': 1, 'think': 1, 'substitute_species': 1, 'relax_structure_m3gnet': 1, 'assess_stability': 1}
────────────────────────────────────────────────────────────
### Ionic Radii Comparison and Prediction
- **Li+ Ionic Radius:** 0.76 Å (octahedral coordination)
- **Na+ Ionic Radius:** 1.02 Å (octahedral coordination)

**Prediction:** The substitution of Li+ with Na+ is expected to cause an **expansion** of the structure due to the larger size of Na+. 

### Potential Issues with Substitution
1. **Increased Strain:** The larger ionic radius of Na+ may introduce strain in the lattice, potentially leading to structural instability.
2. **Volume Change:** A significant volume change could result in mechanical failure, such as cracking or pulverization.
3. **Thermodynamic Stabil

---

## 7. Test Case 3: Perovskite Tolerance Factor

Test understanding of perovskite stability criteria.

In [ ]:
perovskite_query = """
I'm designing a perovskite solar cell absorber.

Starting from CsPbI3 (a known perovskite), I want to substitute:
- Pb with Sn (less toxic)

1. Calculate the Goldschmidt tolerance factor for both CsPbI3 and CsSnI3
   (Use ionic radii: Cs+ = 1.67Å, Pb2+ = 1.19Å, Sn2+ = 1.10Å, I- = 2.20Å)
   Formula: t = (rA + rX) / √2(rB + rX)

2. Predict which will be more stable based on tolerance factor

3. If the structures exist in MP, compare their actual stability (energy above hull)
"""

result3 = run_query(perovskite_query)

---

## 8. Test Case 4: Oxidation State Balance

Test if agent handles charge balance correctly.

In [ ]:
oxidation_query = """
I want to substitute Co³⁺ with Al³⁺ in LiCoO2 to make it cheaper.

1. Is this substitution charge-balanced? Explain.
2. What about substituting Co³⁺ with Mn²⁺? What would happen?
3. Suggest a charge-balanced substitution scheme for Co → Mn

Try the Al substitution (LiCoO2 → LiAlO2) and check stability.
"""

result4 = run_query(oxidation_query)

---

## 9. Test Case 5: Multi-Substitution Exploration

Systematic exploration with stability ranking.

In [ ]:
exploration_query = """
Starting from LiFePO4 (mp-19017), explore these substitutions systematically:

1. Fe → Mn (similar size, same charge)
2. Fe → Co (similar size, same charge)  
3. Fe → Ni (similar size, same charge)

For each:
a) Create the structure
b) Relax with M3GNet
c) Assess stability

Create a summary table ranking them by:
- Energy above hull (stability)
- Volume change from parent (strain indicator)
- Recommendation (promising / marginal / unstable)
"""

result5 = run_query(exploration_query)

---

## 10. Test Case 6: Structure Visualization Request

Explicitly request visualization to test plotting tools.

In [ ]:
viz_query = """
I want to visualize and compare the crystal structures of:
1. LiFePO4 (olivine) - mp-19017
2. LiCoO2 (layered) - mp-22526

Show me:
- The 3D crystal structure of each
- Highlight the Li-ion diffusion pathways
- Explain why olivine has 1D diffusion vs layered has 2D diffusion

Use any visualization tools available.
"""

result6 = run_query(viz_query)

---

## 11. Tool Usage Analytics

In [ ]:
# Show summary of all tools used
show_tool_usage_summary()

# Identify gaps
all_expected = [
    "batch_screen_materials", "analyze_candidate", "generate_phase_diagram",
    "substitute_species", "relax_structure_m3gnet", "assess_stability",
    "plot_structure", "compare_structures", "think"
]

never_used = [t for t in all_expected if TOOL_USAGE_LOG.get(t, 0) == 0]
if never_used:
    print(f"\n⚠️  Tools never called: {never_used}")
    print("   Consider adding test cases that exercise these tools.")

---

## 12. Manual Structure Visualization (Fallback)

If the agent's plotting tools aren't working, here's how to visualize directly.

In [ ]:
# Direct visualization using pymatgen
from mp_api.client import MPRester
from pymatgen.core import Structure

# Fetch a structure
with MPRester() as mpr:
    structure = mpr.get_structure_by_material_id("mp-19017")  # LiFePO4

print(f"Structure: {structure.composition.reduced_formula}")
print(f"Space Group: {structure.get_space_group_info()}")
print(f"Lattice: a={structure.lattice.a:.2f}, b={structure.lattice.b:.2f}, c={structure.lattice.c:.2f} Å")
print(f"Volume: {structure.volume:.2f} Å³")
print(f"Density: {structure.density:.2f} g/cm³")

In [ ]:
# Try interactive 3D visualization (requires crystal_toolkit or py3Dmol)
try:
    import py3Dmol
    
    # Convert to CIF for visualization
    cif_string = structure.to(fmt="cif")
    
    view = py3Dmol.view(width=600, height=400)
    view.addModel(cif_string, "cif")
    view.setStyle({'sphere': {'radius': 0.4}, 'stick': {'radius': 0.1}})
    view.addUnitCell()
    view.zoomTo()
    view.show()
    print("✓ Interactive 3D structure displayed above")
    
except ImportError:
    print("⚠️  py3Dmol not installed. Install with: pip install py3Dmol")
    print("   Falling back to matplotlib...")
    
    # Matplotlib fallback
    try:
        import matplotlib.pyplot as plt
        from pymatgen.analysis.structure_analyzer import SpacegroupAnalyzer
        
        # Get conventional cell for cleaner visualization
        sga = SpacegroupAnalyzer(structure)
        conventional = sga.get_conventional_standard_structure()
        
        fig, ax = plt.subplots(figsize=(8, 6))
        
        # Simple 2D projection
        colors = {'Li': 'green', 'Fe': 'orange', 'P': 'purple', 'O': 'red'}
        for site in conventional:
            elem = str(site.specie)
            ax.scatter(site.coords[0], site.coords[1], 
                      c=colors.get(elem, 'gray'), s=100, label=elem)
        
        ax.set_xlabel('x (Å)')
        ax.set_ylabel('y (Å)')
        ax.set_title(f'{conventional.composition.reduced_formula} - xy projection')
        
        # Remove duplicate labels
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys())
        
        plt.tight_layout()
        plt.savefig(OUTPUT_DIR / "structure_visualization.png", dpi=150)
        plt.show()
        print(f"\n✓ Saved to {OUTPUT_DIR / 'structure_visualization.png'}")
        
    except Exception as e:
        print(f"Visualization failed: {e}")

---

## 13. View LangSmith Traces

If LangSmith is configured, view your traces online.

In [ ]:
if langsmith_enabled:
    project = os.environ.get('LANGCHAIN_PROJECT', 'default')
    print(f"\n🔍 View your traces at:")
    print(f"   https://smith.langchain.com/o/default/projects/p/{project}")
    print("\nLangSmith shows:")
    print("   - Full conversation history")
    print("   - Token usage and costs")
    print("   - Latency per tool call")
    print("   - Error traces for debugging")
else:
    print("\n⚠️  LangSmith not configured.")
    print("To enable, add to your .env file:")
    print("   LANGCHAIN_API_KEY=your-api-key")
    print("   LANGCHAIN_PROJECT=mp-deep-research")
    print("   LANGCHAIN_TRACING_V2=true")
    print("\nGet your API key at: https://smith.langchain.com")

---

## 14. Summary

### Tests Performed

| Test | Focus Area | Tools Expected |
|------|------------|----------------|
| Crystal Analysis | Symmetry, coordination | analyze_candidate |
| Ionic Radius | Substitution logic | substitute_species, relax_m3gnet |
| Perovskite | Tolerance factor | think, batch_screen |
| Oxidation State | Charge balance | substitute_species, assess_stability |
| Multi-Substitution | Systematic exploration | full innovation loop |
| Visualization | Structure plotting | plot_structure, compare_structures |

### Next Steps
1. **Add missing plotting tools** to `tools/__init__.py`
2. **Review LangSmith traces** to identify bottlenecks
3. **Add literature search** to verify "novel" discoveries

In [ ]:
# Final file inventory
print("\n📁 Files created during session:")
for f in sorted(OUTPUT_DIR.glob("*")):
    size = f.stat().st_size / 1024
    print(f"  {f.name} ({size:.1f} KB)")